# **Advanced Model Architectures and Training Techniques**

# **Custom Gaussian Noise Layer**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, mean=0.0, stddev=0.1):
        super(AddGaussianNoise, self).__init__()
        self.mean = mean
        self.stddev = stddev

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(shape=tf.shape(inputs), mean=self.mean, stddev=self.stddev)
            return inputs + noise
        return inputs


# **Simple Model Including the Noise Layer**

In [ ]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize data
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# Define the model
model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    AddGaussianNoise(stddev=0.2),  # Customize stddev as needed
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)


4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/10
750/750 [==============================] - 37s 48ms/step - loss: 0.5317 - accuracy: 0.8084 - val_loss: 0.4608 - val_accuracy: 0.8496
Epoch 2/10
750/750 [==============================] - 32s 43ms/step - loss: 0.3926 - accuracy: 0.8564 - val_loss: 0.3985 - val_accuracy: 0.8739
Epoch 3/10
750/750 [==============================] - 30s 40ms/step - loss: 0.3583 - accuracy: 0.8689 - val_loss: 0.4088 - val_accuracy: 0.8685
Epoch 4/10
750/750 [==============================] - 30s 41ms/step - loss: 0.3377 - accuracy: 0.8734 - val_loss: 0.3680 - val_accuracy: 0.8781
Epoch 5/10
750/750 [==============================] - 29s 39ms/step - loss: 0.3188 - accuracy: 0.8808 - val_loss: 0.3453 - val_accuracy: 0.8843
Epoch 6/10
750/750 [==============================] - 29s 39ms/step - loss: 0.3067 - accuracy: 0.8851 - val_loss: 0.3569 - val_accuracy: 0.8825
Epoch 7/10
750/750 [==============================] - 32s 42ms/step - los

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")


313/313 [==============================] - 3s 10ms/step - loss: 0.3510 - accuracy: 0.8754
Test Accuracy: 0.8754000067710876


# **Residual Block**

In [ ]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, num_filters):
        super(ResidualBlock, self).__init__()
        self.conv1 = keras.layers.Conv2D(num_filters, kernel_size=3, padding='same', activation='relu')
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = keras.layers.Conv2D(num_filters, kernel_size=3, padding='same', activation=None)
        self.bn2 = keras.layers.BatchNormalization()
        self.relu = keras.layers.Activation('relu')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += inputs  # Adding the input to the output of the block
        x = self.relu(x)
        return x


# **Model Using the Residual Block**

In [ ]:
# Define the model
model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    ResidualBlock(32),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    ResidualBlock(32),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)


Epoch 1/10
750/750 [==============================] - 275s 364ms/step - loss: 0.6748 - accuracy: 0.7895 - val_loss: 0.5769 - val_accuracy: 0.7939
Epoch 2/10
750/750 [==============================] - 274s 365ms/step - loss: 0.3671 - accuracy: 0.8756 - val_loss: 0.5685 - val_accuracy: 0.7814
Epoch 3/10
750/750 [==============================] - 266s 355ms/step - loss: 0.3119 - accuracy: 0.8913 - val_loss: 0.4115 - val_accuracy: 0.8540
Epoch 4/10
750/750 [==============================] - 278s 371ms/step - loss: 0.2836 - accuracy: 0.8989 - val_loss: 0.3276 - val_accuracy: 0.8854
Epoch 5/10
750/750 [==============================] - 266s 355ms/step - loss: 0.2593 - accuracy: 0.9078 - val_loss: 0.3286 - val_accuracy: 0.8823
Epoch 6/10
750/750 [==============================] - 265s 354ms/step - loss: 0.2468 - accuracy: 0.9128 - val_loss: 0.3388 - val_accuracy: 0.8801
Epoch 7/10
750/750 [==============================] - 260s 346ms/step - loss: 0.2333 - accuracy: 0.9163 - val_loss: 0.4751 -

# **Evaluate the Model**

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")


313/313 [==============================] - 13s 42ms/step - loss: 0.3156 - accuracy: 0.8933
Test Accuracy: 0.8932999968528748


## **AddGaussianNoise Layer**

In [ ]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, mean=0.0, stddev=0.1):
        super(AddGaussianNoise, self).__init__()
        self.mean = mean
        self.stddev = stddev

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(shape=tf.shape(inputs), mean=self.mean, stddev=self.stddev)
            return inputs + noise
        return inputs


## **Model Using the AddGaussianNoise Layer**

In [ ]:
# Load Fashion MNIST data
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# Define the model
model = keras.Sequential([
    AddGaussianNoise(0.0, 0.1),
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)


Epoch 1/10
750/750 [==============================] - 8s 9ms/step - loss: 0.6214 - accuracy: 0.7800 - val_loss: 0.4328 - val_accuracy: 0.8419
Epoch 2/10
750/750 [==============================] - 8s 10ms/step - loss: 0.4690 - accuracy: 0.8314 - val_loss: 0.3971 - val_accuracy: 0.8578
Epoch 3/10
750/750 [==============================] - 6s 8ms/step - loss: 0.4356 - accuracy: 0.8408 - val_loss: 0.3883 - val_accuracy: 0.8557
Epoch 4/10
750/750 [==============================] - 7s 9ms/step - loss: 0.4116 - accuracy: 0.8501 - val_loss: 0.3744 - val_accuracy: 0.8632
Epoch 5/10
750/750 [==============================] - 6s 7ms/step - loss: 0.3970 - accuracy: 0.8549 - val_loss: 0.3609 - val_accuracy: 0.8722
Epoch 6/10
750/750 [==============================] - 7s 9ms/step - loss: 0.3848 - accuracy: 0.8603 - val_loss: 0.3600 - val_accuracy: 0.8673
Epoch 7/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3821 - accuracy: 0.8619 - val_loss: 0.3534 - val_accuracy: 0.8700
Epoch

# **Evaluate the Model**

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")


313/313 [==============================] - 1s 3ms/step - loss: 0.3845 - accuracy: 0.8640
Test Accuracy: 0.8640000224113464


# **custom training loop**

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Load Fashion MNIST data
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize and reshape data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(10)  # No activation function as logits will be output
])

# Although using a custom loop, still need to compile for the evaluation
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Define training parameters
batch_size = 64
epochs = 10
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024).batch(batch_size)

# Custom training loop
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = keras.losses.sparse_categorical_crossentropy(y_batch_train, logits, from_logits=True)

        grads = tape.gradient(loss_value, model.trainable_weights)
        model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 200 == 0:
            print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss_value.numpy().mean())))

# Evaluate the model on the test set
test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])



Start of epoch 0
Training loss (for one batch) at step 0: 2.8375
Training loss (for one batch) at step 200: 0.4221
Training loss (for one batch) at step 400: 0.2709
Training loss (for one batch) at step 600: 0.3404
Training loss (for one batch) at step 800: 0.5491

Start of epoch 1
Training loss (for one batch) at step 0: 0.3692
Training loss (for one batch) at step 200: 0.2799
Training loss (for one batch) at step 400: 0.5351
Training loss (for one batch) at step 600: 0.3936
Training loss (for one batch) at step 800: 0.4826

Start of epoch 2
Training loss (for one batch) at step 0: 0.1773
Training loss (for one batch) at step 200: 0.2271
Training loss (for one batch) at step 400: 0.2805
Training loss (for one batch) at step 600: 0.3208
Training loss (for one batch) at step 800: 0.2764

Start of epoch 3
Training loss (for one batch) at step 0: 0.2591
Training loss (for one batch) at step 200: 0.4343
Training loss (for one batch) at step 400: 0.3098
Training loss (for one batch) at ste